# Floquet instability

This notebook discusses how we calculate the Floquet exponents for symmetric and asymmetric inflation potentials, using $\alpha$-attractor T- and E-model potentials as examples.

In [1]:
import numpy as np
import scipy.interpolate as spp
from numpy.lib.scimath import sqrt as sqrt
import scipy as sp
import scipy.integrate as spi
import scipy.interpolate as spp
import scipy.optimize as spo
import matplotlib.pyplot as plt

In [3]:
# Defining Parameters (Following are in GeV)
# If the quantites you have are already dimensionless, make the appropriate changes here

lam = 100*(1/6)**0.5                   # The coupling parameter
m2 = 1.6799e+28                    # Mass of the field squared
m = m2**0.5                        # Mass of the field
mp = 2.435e18                      # reduced Planck Mass
g2 = 8e-6

# Converting them into dimensionless quantities (w.r.t. m)              
m = m/mp
V0 = 8.5000e-13                    # V_0/mp^4


# Constructing arrays
n = 250
x0 = np.linspace(0.001, 0.06, n)
k = np.linspace(0.1, 2.5, n)

q = 0.25*g2*(x0/m)**2.0
Ak = k**2.0 + 2.0*q

# print(m)
# print(Ak)
# print(q)

savefile = 'Flq_Exp_Tmodel_100sqrt(16)_8.0e-6.txt'

Defining the potentials

$$ V_{\rm{E}} = V_0 \left(1 - e^{-\lambda \phi}\right)^2 \quad\quad V_{\rm{T}} = V_0\tanh^2\left(\lambda\phi\right) $$
and their derivatives $V_{,\phi}$ and $V_{,\phi\phi}$

In [4]:
# Defining Inflationary Potential 
# Write the potential and its first, second derivatives here 

# E-model (Disable when not in use)
def V(xx, lam = lam):
    return V0*(1 - np.exp(-lam*xx))**2.0

def dV(xx, lam = lam):
    return 2.0*V0*lam*np.exp(-2.0*lam*xx)*(np.exp(lam*xx) - 1)

def d2V(xx, lam = lam):
    return -2.0*V0*(lam**2.0)*np.exp(-2.0*lam*xx)*(np.exp(lam*xx) - 2)

# T-model (Disable when not in use)
# def V(xx, lam = lam):
#     return V0*np.tanh(lam*xx)**2.0

# def dV(xx, lam = lam):
#     return 2.0*V0*lam*np.tanh(lam*xx)*np.power(np.cosh(lam*xx), -2.0)

# def d2V(xx, lam = lam):
#     return 2.0*V0*(lam**2.0)*(np.power(np.cosh(lam*xx), -4.0) - 2.0*np.power(np.tanh(lam*xx), 2.0)*np.power(np.cosh(lam*xx), -2.0))

# Quadratic Potential (Disable when not in use)
# def V(xx, mm = m):
#     return 0.5*(mm**2.0)*(xx**2.0)

# def dV(xx, mm = m):
#     return (mm**2.0)*xx

# def d2V(xx, mm = m):
#     return mm**2.0

The oscillation period calculation depends on whether the potential is symmetric or asymmetric. In general, we can write the period $T$ as 
$$ T = 2\int_0^{\phi_i}\frac{d\phi}{\sqrt{2V(\phi_i)-2V(\phi)}}+\int_{\phi_f}^0\frac{d\phi}{\sqrt{2V(\phi_f)-2V(\phi)}} = T_{(+)}+T_{(-)} $$
where $\phi_i$ and $\phi_f$ are the amplitudes of the inflaton on the right and left sides of the potential respectively. When $V(\phi)=V(-\phi)$, we have $\phi_i=\phi_f$.

In [5]:
# Calculating the Time-period of the Oscillation

# Finding the points where V = V(x0)
def fldvls(x0, lam):
    Vend = V(x0, lam)
    if V(-x0, lam) == Vend:
        return [-x0, x0]
    else:
        def roots(xx, x0, lam = lam):
            return V(xx, lam) - V(x0, lam)
        rts = spo.root_scalar(roots, args=(x0, lam), bracket=[-x0, 0])
        return [rts.root, x0] 
    
def timeintegrand(xx, xlim, lam = lam):
    return (2.0*V(xlim, lam) - 2.0*V(xx, lam))**(-0.5)

def TimePeriod(x0, lam = lam):
    x_minus, x_plus = fldvls(x0, lam)
    T_minus = spi.quad(timeintegrand, x_minus, 0, args = (x_minus, lam), epsabs = 10e-10)[0]
    T_plus = spi.quad(timeintegrand, 0, x_plus, args = (x_plus, lam), epsabs = 10e-10)[0]

    return 2.0*(T_minus + T_plus)

The Floquet exponents are complex-valued, in general. For the study of growth of instabilities, we are interested in the real part of the exponent $\Re(\mu_k)$, which is calculated as
$$ \Re(\mu_k)=\frac{1}{T}\ln\left[ \frac{1}{2}\left( \delta\phi_k^{(1)}+\dot{\delta\phi}_k^{(2)} \right)\pm\frac{1}{2}\sqrt{ \left( \delta\phi_k^{(1)}-\dot{\delta\phi}_k^{(2)} \right)^2 +4\delta\phi_k^{(2)}\dot{\delta\phi}_k^{(1)} }\right] $$
using orthogonal initial conditions $\{ \delta\phi_k^{(1)}(0)=1,\dot{\delta\phi}_k^{(1)}(0)=0 \}$ and $\{ \delta\phi_k^{(2)}(0)=0,\dot{\delta\phi}_k^{(2)}(0)=1 \}$.

In [6]:
# Finding Floquet Exponents
def flqexp(k, x0, T, lam = lam):
    def fun(var, t):
        [xp, yp, dx, dy] = var
        xpt = yp
        dxt = dy
        ypt = -dV(xp, lam)*(m**-2.0)
        # dyt = -(k**2.0 + d2V(xp, lam)*(m**-2.0))*dx
        dyt = -(k**2.0 + g2*xp**2.0*(m**-2.0))*dx
        return [xpt, ypt, dxt, dyt]

    Time = np.linspace(0, m*T, 80000)
    sol1 = spi.odeint(fun, [x0, 0, 1, 0], Time, rtol = 3e-14,  atol = 2e-35, mxstep = 900000000)
    sol2 = spi.odeint(fun, [x0, 0, 0, 1], Time, rtol = 3e-14,  atol = 2e-35, mxstep = 900000000)

    x1, y1, dx1, dy1 = sol1.T
    x2, y2, dx2, dy2 = sol2.T

    mu_plus = (1/T)*np.log(np.abs(0.5*(dx1 + dy2) + 0.5*sqrt((dx1 - dy2)**2.0 + 4.0*dx2*dy1)))
    mu_minus = (1/T)*np.log(np.abs(0.5*(dx1 + dy2) - 0.5*sqrt((dx1 - dy2)**2.0 + 4.0*dx2*dy1)))

    mu_plus = mu_plus[-1]
    mu_minus = mu_minus[-1]

    mu = np.maximum(mu_plus, mu_minus)/m

    return [k, x0, mu]

mu = np.zeros((n, n))
for i in range(n):
    print(i)
    T = TimePeriod(x0[i])
    for j in range(n):
        mu[i, j] = flqexp(k[j], x0[i], T)[2]

np.savetxt(savefile, mu)
print('Done')


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
Done


In [ ]:
# DIY mode
x0 = 0.003
T = TimePeriod(x0, lam)
print(m*T)
k = 0.1       # k/m
def fun(var, t):
    [xp, yp, dx, dy] = var
    xpt = yp
    dxt = dy
    ypt = -dV(xp, lam)*(m**-2.0)
    dyt = -(k**2.0 + d2V(xp, lam)*(m**-2.0))*dx
    return [xpt, ypt, dxt, dyt]

Time = np.linspace(0, m*T, 80000)
sol1 = spi.odeint(fun, [x0, 0, 1, 0], Time, rtol = 3e-14,  atol = 2e-35, mxstep = 900000000)
sol2 = spi.odeint(fun, [x0, 0, 0, 1], Time, rtol = 3e-14,  atol = 2e-35, mxstep = 900000000)

x1, y1, dx1, dy1 = sol1.T
x2, y2, dx2, dy2 = sol2.T

# y1, y2, dy1, dy2 = m*y1, m*y2, m*dy1, m*dy2

mu_plus = np.log(np.abs(0.5*(dx1 + dy2) + 0.5*sqrt((dx1 - dy2)**2.0 + 4.0*dx2*dy1)))
mu_minus = np.log(np.abs(0.5*(dx1 + dy2) - 0.5*sqrt((dx1 - dy2)**2.0 + 4.0*dx2*dy1)))

mu_plus = mu_plus[-1]
mu_minus = mu_minus[-1]

mu = np.maximum(mu_plus, mu_minus)/(m*T)
print(mu)
plt.plot(Time, x1)
plt.plot(Time, y1)
# plt.plot(Time, dx1)
# plt.axhline(0.0, color='k')